# Constraining population in benzene in RHF

In this notebook I will try to constrain the electronpopulation on 1 carbon atom like I did in the Hückel model. This time I will use RHF for this proces. Because I am intrested in behaviors of $\pi$-systems I will start with Benzene. 

In [ ]:
import numpy as np
from sympy import symbols, Matrix, simplify
import matplotlib.pyplot as plt
from pyscf import gto, scf, ao2mo
import sys
sys.path.append('/workspaces/Huckel-on-benzene/project/Benzene/rhf')
import plottingsystem as pts
benzene_molecule = '''
  C    1.2116068    0.6995215    0.0000000
  C    1.2116068   -0.6995215    0.0000000
  C   -0.0000000   -1.3990430   -0.0000000
  C   -1.2116068   -0.6995215   -0.0000000
  C   -1.2116068    0.6995215   -0.0000000
  C    0.0000000    1.3990430    0.0000000
  H    2.1489398    1.2406910    0.0000000
  H    2.1489398   -1.2406910    0.0000000
  H   -0.0000000   -2.4813820   -0.0000000
  H   -2.1489398   -1.2406910   -0.0000000
  H   -2.1489398    1.2406910   -0.0000000
  H    0.0000000    2.4813820    0.0000000
'''

extra_bound = [(0,5)]
carbon_coordinates = pts.taking_carbon_coordinates(benzene_molecule)
print(carbon_coordinates)
benzene_molecule_with_H = gto.M(atom=benzene_molecule, basis='sto-3g')#'sto-6g''ccpvdz'

pz_indices = [i for i, label in enumerate(benzene_molecule_with_H.ao_labels()) if 'C 2pz' in label]
print(pz_indices)
hf = scf.RHF(benzene_molecule_with_H)
hf_energy = hf.kernel()

# Extract integrals
H_core = hf.get_hcore()
S = hf.get_ovlp()
eri = ao2mo.restore(1, hf._eri, benzene_molecule_with_H.nao_nr())


In [ ]:

def build_fock_matrix(H_core, P, eri):
    F = H_core.copy()
    for p in range(H_core.shape[0]):
        for q in range(H_core.shape[1]):
            F[p, q] += np.sum(P * (eri[p, q] - 0.5 * eri[p,:,q,:]))
    return F

In [ ]:
def apply_lagrange_multipliers(F, constraints, lambdas):
    for mu in range(len(constraints)):
        C_mu = constraints[mu]
        F += lambdas[mu] * np.outer(C_mu, C_mu)
    return F

In [ ]:
def scf_cycle(H_core, S, eri, constraints, lambdas, target_pop, max_iter=100, tol=1e-6):
    # Initialisatie
    P = np.zeros_like(H_core)
    energy_old = 0.0

    for iteration in range(max_iter):
        # Bouw de Fock matrix
        F = build_fock_matrix(H_core, P, eri)

        # Pas Lagrange multipliers toe
        F = apply_lagrange_multipliers(F, constraints, lambdas)

        # Diagonaliseer de Fock matrix
        eigvals, C = np.linalg.eigh(F)

        # Bereken de nieuwe dichtheidsmatrix
        P_new = np.zeros_like(P)
        for i in range(len(eigvals)//2):  # Voor RHF nemen we de helft van de orbitalen
            P_new += 2 * np.outer(C[:, i], C[:, i])

        # Bereken de elektronische energie
        energy_new = np.sum(P_new * (H_core + F)) / 2

        # Controleer de elektronenpopulatie op de specifieke koolstof
        current_pop = np.sum(P_new * constraints[0])
        lambda_adjustment = lambdas[0] + (target_pop - current_pop)
        lambdas[0] = lambda_adjustment

        # Controleer de convergentie
        if np.abs(energy_new - energy_old) < tol and np.abs(target_pop - current_pop) < tol:
            print(f"SCF converged in {iteration+1} iterations.")
            break

        # Update de dichtheidsmatrix en energie
        P = P_new
        energy_old = energy_new

    # Energiecorrectie voor de Lagrange multipliers
    correction = sum(l * np.sum(P_new * C) for l, C in zip(lambdas, constraints))/2
    corrected_energy = energy_new - correction

    return corrected_energy, C, eigvals, lambdas

In [ ]:
constraints = [np.zeros(H_core.shape[0])]
constraints[0][0] = 1.0 
lambdas = [0.1]
target_pop = 1


energy, C, eigvals, lambdas = scf_cycle(H_core, S, eri, constraints, lambdas, target_pop)
print("Gecorrigeerde Energie:", energy)
print("Moleculaire orbitalen coefficients:\n", C)
print("Orbitalen energieën:\n", eigvals)
print("Lagrange multipliers:\n", lambdas)


In [ ]:
pops = np.linspace(0.1, 1.9, 19)
print(pops)
for pop in pops:
    energy, C, eigvals, lambdas = scf_cycle(H_core, S, eri, constraints, lambdas, pop)
    print(energy)